<font size=8><center> API - Preprocessors & Predictions</center></font>

Goal - create a function that: 
- accepts a dictionary of byte files in format {"filename":bytes}
- create "filenames" list of dict keys
- creates "tiles" array
- gets images from byte files
- resizes images to (675,675,3), tiles into 9
- converts tiles to arrays and saves in "tiles" array in format [[9 x NumPy arrays],[9 x NumPy arrays]...#num of images]
- resizes images to (225,225,3) and converts to NumPy array
- reshapes image array to tensor of shape (-1,225,225,3)
- appends tensor to list of tensors
- returns list of tensors of shape (-1,225,225,3)

Tile preprocessing for Model 2
- recieves from Model 1 a list of indexes(?) for Model 2 input data
- store list of indexes (Important - needed to identify original file!)
- for each index i in "tiles" array, converts each of the 9 x tiles into tensors of shape (-1,225,225,3)
- appends array at index i to list of arrays
- returns list of tensors of shape (-1,225,225,3)
  


## Imports

In [21]:
#pip install tensorflow
#pip install OpenCV

from io import BytesIO
import numpy as np
from tensorflow.keras.utils import img_to_array
from PIL import Image
import json
import cv2

from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

## Load data

In [50]:
try:
    with open("/Users/leila/code/meloeckert/solar_project/api/api-test-data/Cleaan (1).jpeg", "rb") as f:
        bytes = f.read()
    
except IOError:
     print('Error while opening the file')  

In [51]:
try:
    with open("/Users/leila/code/meloeckert/solar_project/api/api-test-data/Dust (1).jpg", "rb") as f:
        bytes_1 = f.read()
    
except IOError:
     print('Error while opening the file')  

In [54]:
test = {"img":bytes,
        "img2":bytes_1,
           }

## Preprocessing

In [93]:
def preprocess(images : dict, num_tiles=3):
    """
    Predict presence of damage and damage class based on image
    """
    filenames=list(images.keys())
    tile_arrays=[]
    tensors=[]
    invalid={}
    
    for f in filenames:
    
        img = images.get(f'{f}')
        img = Image.open(io.BytesIO(img))
        try:

            #image tiling, array conversion; append to 'tile_arrays' list
            image_tiles = []
            img_675_array = np.array(img.resize((675,675)))

            #vertical slice; num_tiles = # of equal divisions
            for v in range(0,img_675_array.shape[0],img_675_array.shape[0]//num_tiles):
                #horizontal slice; num_tiles = # of equal divisions
                for h in range(0,img_675_array.shape[1],img_675_array.shape[1]//num_tiles):
                    tile_array = img_675_array[v:v+(img_675_array.shape[0]//num_tiles), h:h+(img_675_array.shape[1]//num_tiles),:]
                    image_tiles.append(tile_array) 
            
            tile_arrays.append(image_tiles)

            #resize full image and convert to array; append to 'tensors' list
            img_255_array = np.array(img.resize((225,225)))
            tensor = img_255_array.reshape((-1, 225, 225, 3))            
            tensors.append(tensor)
        
        except Exception as e:
            if f.lower().endswith("ds._store"):
                continue
            else:
                print("Error processing: ", f)
                print("Error: ", e)
                invalid[f]= e                    
                continue

    preprocessed_X = {"filenames":filenames,
                     "tensors":tensors,
                     "tile_arrays":tile_arrays, 
                      "invalid":invalid
                     }
        
    return preprocessed_X 

In [ ]:
for r in range(0,img.shape[0],30):
    for c in range(0,img.shape[1],30):
        cv2.imwrite(f"img{r}_{c}.png",img[r:r+30, c:c+30,:])

In [91]:
X_prep = preprocess(test)
len(X_prep['tile_arrays'][0])

25

In [8]:
def preprocess_tiles(image_indexes_model_2 : list):
    """
    Predict presence of damage and damage class based on image
    """
    preprocessed_X_tiles=[]
    for i in image_indexes_model_2:
        tile_tensors = []
        for t in tile_arrays[i]:
            tile_tensors.append(array_to_tensor(t))
        preprocessed_X_tiles.append(tile_tensors)

    return preprocessed_X_tiles

## Load model

In [42]:
model = load_model('/Users/leila/code/meloeckert/solar_project/model_multiclass_clean_damage_dirt.h5')

In [43]:
def find_index_of_max_element(input_list):
    max_value = max(input_list)
    max_index = input_list.index(max_value)
    return max_index

In [92]:
#app.state.model = load_model()

def predict_1(preprocessed_X : dict):
    
    results = {}
    names_of_classes = ['clean','damaged','dirty']
    
    for x in range(len(preprocessed_X['tensors'])):
        preds = model.predict(preprocessed_X['tensors'][x])
        res = names_of_classes[find_index_of_max_element(preds[0].tolist())]
        
        if res != 'dirty':
            res = predict_2(preprocessed_X['tile_arrays'][x])
              
        filename = preprocessed_X['filenames'][x]
        res[f'{filename}'] = res
        
    results_json = json.dumps(results)
    return results_json

In [72]:
preprocessed_X = preprocess(test)
predict_1(preprocessed_X))

In [ ]:
def predict_2(tile_arrays : dict):
    results = {}

    for t in tile_arrays:
        preds = model_2.predict(t)
        res = names_of_classes[find_index_of_max_element(preds[0].tolist())]
        results.append(res)

    #some logic to derive single result from tile results
    #if using pred probabilities, amend for-loop to append preds somewhere

    res = "tbd"
    return res

## Rough work   

In [ ]:

        
    
    
    def convert_to_jpeg(filepath):
        #Load image
        img= Image.open(filepath)
        
        # Convert the image to RGB mode
        img = img.convert("RGB")
                
        # Save the image in JPEG format as <original filepath>.jpeg to the "jpeg_tmp" folder 
        name = os.path.basename(filepath).replace("jpg", "jpeg")
        new_filepath = os.path.join(os.getcwd(), "jpeg_tmp", name)
        img.save(new_filepath, "JPEG")
                
        # Close the image to free resources
        img.close()
        
        #print("Image format conversion complete.")     
        return new_filepath

    
    def file_to_arrays_list(filepath):
        '''Converts JPEG files to arrays and appends arrays & filenames to 'arrays' & 'filenames' lists
        Appends filenames of unprocessible files to 'invalid' list'''
        
        image= Image.open(filepath)
        image_array=np.array(image)
        arrays.append(image_array)


    filepaths = []
    arrays=[]
    invalid = []
    
    #Gets file paths for input files
    for root, dirs, files in os.walk(image_path):
        for file in files:
            filepath = os.path.join(root, file)
            #Ignores macOS hidden system file in directories
            if filepath.lower().endswith(".ds_store"):
                continue
            
            try:
                #Converts JPEG files to arrays and add filepath to 'filepaths' list
                if file.lower().endswith(".jpeg"):
                    file_to_arrays_list(filepath)
                    filepaths.append(filepath)
                
                #Processes non-JPEG image files, converts to arrays and adds original image filepath to 'filepaths' list
                else:           
                    #Creates jpeg_tmp for converted images if not already created                        
                    if not os.path.exists(os.path.join(os.getcwd(), "jpeg_tmp")):
                            os.makedirs(os.path.join(os.getcwd(), "jpeg_tmp"))
                
                    #Save JPEG to jpeg_tmp with original filepath as name              
                    original_filepath=filepath
                    filepath = convert_to_jpeg(filepath)
                    
                    #Converts JPEGs to arrays and adds original image filepath to 'filepaths' list
                    file_to_arrays_list(filepath)
                    filepaths.append(original_filepath)
                    os.remove(filepath)
            
            except Exception as e:
                if file.lower().endswith(".DS_Store"):
                    continue
                else:
                    print("Error processing:", filepath)
                    print("Error:", e)
                    invalid.append(filepath)
                    if filepath.lower().endswith(".ds_store"):
                        print("Skipping .DS_Store:", filepath)   
                        print(filepath)
                    

    print("Filepaths list is "+str(len(filepaths)))
    print("Arrays list"+str(len(arrays)))
    print("Invalid is "+str(len(invalid)))
    
    return {'Filename':filepaths, 
        'Prediction': arrays,
           'Errors': invalid}